In [1]:
1

1

In [2]:
using Mongoc
using Dates

In [3]:
using JSON
using JLD2
using HDF5
using Profile

In [4]:
using Dates
using Tables
using DataFrames
using CSV
using XLSX
using BSON
using Distributed
using FileIO
# using Printf
# using Query

addprocs()

32-element Vector{Int64}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
  ⋮
 22
 23
 24
 25
 26
 27
 28
 29
 30
 31
 32
 33

In [5]:
using DataFramesMeta

In [6]:
f_mst = "../dicts/Jm01_dis_parse/byomei_cancer_20210701.csv"

"../dicts/Jm01_dis_parse/byomei_cancer_20210701.csv"

In [7]:
df_mst = CSV.read(f_mst, header=0,types=Dict(4=>String), DataFrame)
const dis_codes = Set(df_mst.Column4)

Set{String} with 3440 elements:
  "2312002"
  "2386002"
  "2395024"
  "8846705"
  "8838904"
  "8846604"
  "1892003"
  "2159040"
  "8837256"
  "8837929"
  "8847309"
  "8846681"
  "8847161"
  "8846816"
  "8848610"
  "8849418"
  "1910003"
  "8846359"
  "8848712"
  "1579002"
  "8848820"
  "1800003"
  "1919004"
  "8847917"
  "2059002"
  ⋮ 

In [8]:
first(df_mst, 3)

Row,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12,Column13,Column14,Column15,Column16,Column17,Column18,Column19,Column20,Column21,Column22,Column23,Column24,Column25,Column26,Column27,Column28,Column29,Column30,Column31,Column32,Column33,Column34,Column35,Column36,Column37,Column38,Column39,Column40,Column41,Column42,Column43,Column44,Column45,Column46,Column47,Column48,Column49,Column50,Column51
,Int64,Int64,String1,String,Int64,Int64,String,Int64,String,Int64,String,Int64,Int64,String7,String3,String3,String7,String7?,String3,Int64,Int64,Int64,String15,String15,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,String3,String,String3,String3,Int64,Int64,Int64,Int64,Int64,String1,Int64,Int64,Bool
1,1,0,B,8847269,8847269,18,１系統に異形成を伴う不応性血球減少症,18,１系統に異形成を伴う不応性血球減少症,34,１ケイトウニイケイセイヲトモナウフオウセイケッキュウゲンショウショウ,20098096,1,AD6V,NA,NA,D460,missing,NA,0,0,0,20121001,NA,99999999,0,0,0,0,0,0,0,0,0,0,0,0,0,NA,１系統に異形成を伴う不応性血球減少症,NA,NA,18,0,0,0,0,D,460,3,true
2,2,0,B,8847270,8847270,6,５ｑ－症候群,6,５ｑ－症候群,13,５ｑマイナスショウコウグン,20098113,1,S1E9,NA,NA,D466,missing,NA,0,0,0,20121001,NA,99999999,0,0,0,0,0,0,0,0,0,0,0,0,0,NA,５ｑ－症候群,NA,NA,6,0,0,0,0,D,466,3,true
3,3,0,B,8847271,8847271,15,ＡＬＫ陰性未分化大細胞リンパ腫,15,ＡＬＫ陰性未分化大細胞リンパ腫,22,ＡＬＫインセイミブンカダイサイボウリンパシュ,20098093,1,KS53,NA,NA,C847,missing,NA,0,0,5,20121001,NA,99999999,0,0,0,0,0,0,0,0,0,0,0,0,0,NA,ＡＬＫ陰性未分化大細胞リンパ腫,NA,NA,15,0,0,0,0,C,847,3,true


In [9]:
f_iymst = "../dicts/Jm01_dis_parse/dfmeta_iy_20231122_150053_929635.csv"
df_iymst = CSV.read(f_iymst, header=1,types=Dict("c2"=>String), DataFrame)
# const dis_codes = Set(df_mst.Column4)
first(df_iymst, 3)

Row,Column1,c0,c2,c4,c9,c10,c13,c14,c15,c16,c31,ymd,next_month_day1,next_month_day2,filename,dt_rank_neg
,Int64,Int64,String,String,String15,Float64?,Float64,Float64,Float64,Float64,String15,Date,Date,Date,String31,Float64
1,2802,9,610406002,アストニール錠１０ １０ｍｇ,錠,missing,0.0,0.0,0.0,1.0,2123014F2031,2013-03-31,2013-04-01,2013-04-02,y20130329.csv,1.0
2,3553,5,610406008,アゼピット錠１ｍｇ,錠,missing,0.0,0.0,0.0,1.0,4490004F2010,2013-09-30,2013-10-01,2013-10-02,y20130611.csv,1.0
3,52035,9,610406009,アセメール錠１０ １０ｍｇ,錠,1.0,0.0,0.0,0.0,1.0,2123014F2040,2014-04-01,2014-05-01,2014-05-02,y20140305.csv,1.0


In [10]:
df_iymst.c9[1]

"錠"

In [11]:
df_drg_tab = @chain df_iymst begin
    @subset(:c9 .== "錠")
end

const tab_codes = Set(df_drg_tab.c2)

Set{String} with 12936 elements:
  "620703401"
  "612430006"
  "612590038"
  "610443003"
  "621521203"
  "622907901"
  "620005051"
  "610406015"
  "622556801"
  "610412050"
  "610412077"
  "622668001"
  "622031401"
  "622147503"
  "613120002"
  "622537801"
  "622604301"
  "622295201"
  "620008464"
  "622077701"
  "622718700"
  "622718900"
  "610454012"
  "622389001"
  "622011801"
  ⋮ 

In [12]:
client = Mongoc.Client("192.168.0.25", 27017)
db = client["ndb"]
collection = db["d_000"]

Collection(Database(Client(URI("mongodb://192.168.0.25:27017")), "ndb"), "d_000")

In [13]:
# dbs = ["d_" * lpad(i, 3, '0') for i in 0:255]

const dbs = ["d_" * lpad(i, 3, '0') for i in 0:255]

256-element Vector{String}:
 "d_000"
 "d_001"
 "d_002"
 "d_003"
 "d_004"
 "d_005"
 "d_006"
 "d_007"
 "d_008"
 "d_009"
 "d_010"
 "d_011"
 "d_012"
 ⋮
 "d_244"
 "d_245"
 "d_246"
 "d_247"
 "d_248"
 "d_249"
 "d_250"
 "d_251"
 "d_252"
 "d_253"
 "d_254"
 "d_255"

In [14]:
collection = db["d_000"]

Collection(Database(Client(URI("mongodb://192.168.0.25:27017")), "ndb"), "d_000")

In [15]:
Mongoc.ping(client)

BSON("{ "ok" : 1.0 }")

In [16]:
# docs = Mongoc.find_one(collection, )

In [17]:
a = now()

2024-10-15T13:33:55.647

In [18]:
b = now()
print(canonicalize(b -a))

466 milliseconds

In [19]:
a = now()
l = []
# i = 0
for (i,e) in enumerate(collection)
    # i += 1
    
    if i % 1e+5 == 0
        b = now()
        print(i)
        print("\n")
        print(canonicalize(b -a))
        print("\n")
        a = now()
    end

    if i == 1000
        break
    end
end

In [20]:
function check_ex(e)
    len_sch = length(e["dat1"])
    println("lensch:"* string(len_sch))
    for i in 1:len_sch
        sch = e["dat1"][i]["dat2"]
        println("sch::"*string(i))
        len_tbs = length(sch)
        for j in 1:len_tbs
            j_lab = sch[j]["tb"]
            tb = sch[j]["dat3"]
            print(string(j)*":")
            print(sch[j]["tb"]*", ")
            x = d2skvvd(tb)
            
        end
        println("")
    end
end

check_ex (generic function with 1 method)

In [21]:
function get_schs(e)
    Dict(e["dat1"])
end

get_schs (generic function with 1 method)

In [22]:
const col_days = ["days"* lpad(i, 2, '0') *"_info" for i in 1:31]
col_days

31-element Vector{String}:
 "days01_info"
 "days02_info"
 "days03_info"
 "days04_info"
 "days05_info"
 "days06_info"
 "days07_info"
 "days08_info"
 "days09_info"
 "days10_info"
 "days11_info"
 "days12_info"
 "days13_info"
 ⋮
 "days20_info"
 "days21_info"
 "days22_info"
 "days23_info"
 "days24_info"
 "days25_info"
 "days26_info"
 "days27_info"
 "days28_info"
 "days29_info"
 "days30_info"
 "days31_info"

In [23]:
const tb_short = Set(["iy", "to", "si"])

Set{String} with 3 elements:
  "si"
  "to"
  "iy"

In [24]:
const tb_dis = Set(["iy", "to", "si"])
const tb_re = Set(["re"])

Set{String} with 1 element:
  "re"

In [25]:
# const tb3_pha = Set(["cz", "iy", "sh"])
const tb3_pha = Set(["cz", "iy"])

Set{String} with 2 elements:
  "cz"
  "iy"

In [26]:
const cols_days_s = Set{String}(col_days)

Set{String} with 31 elements:
  "days10_info"
  "days09_info"
  "days11_info"
  "days29_info"
  "days02_info"
  "days05_info"
  "days12_info"
  "days20_info"
  "days27_info"
  "days13_info"
  "days28_info"
  "days01_info"
  "days03_info"
  "days04_info"
  "days15_info"
  "days16_info"
  "days21_info"
  "days08_info"
  "days23_info"
  "days17_info"
  "days22_info"
  "days06_info"
  "days25_info"
  "days31_info"
  "days24_info"
  ⋮ 

In [27]:
# const s = col_days
# ss_need_cols
# cols_days_s
# tb_short
# need_cols
# col_days
# tb_dis
# tb_re
# tb3_pha

In [28]:
need_cols = [
    "times",
    "medicine_cd",
    "use_amnt",
    "value",
    "prac_act_cd",
    "qnt_dat",
    "spcfc_dev_cd",
    "days",
    "aft_splmt_prac_ident",
    "seq2_no",
    "skwd_name_cd",
    "sspct_dss_flg",
    "sex_div",
    "age_hier_cd1",
    "oprtn_ymd",
    "rcpt_comp_sys_rec",
    "prac_ident",
    "prspt_no",
    "dose",
    "dspng_amnt",
    "prspt_ymd",
    "dspng_ymd"
]
const ss_need_cols = Set{String}(vcat(need_cols, col_days))
length(need_cols)

22

In [1]:
# e0 = Mongoc.find_one(collection,)
# e = Dict(e0)

In [30]:
# qs = Mongoc.BSON("""{"ym":"201104"}""")
qs = Mongoc.BSON("""{"ym":"202104"}""")

BSON("{ "ym" : "202104" }")

In [31]:
es = []
enum = 10000
for (i, doc) in enumerate(Mongoc.find(collection, qs))
    # println(i)
    push!(es,doc)
    if i == enum
        break
    end
# for i in 1:100
#     println(i)
#     push!(es,Mongoc.find(collection, qs))
end

In [1]:
# e = Dict(es[1])

In [33]:
# e["dat1"]

In [34]:
1 in [1,2,3]

true

In [35]:
tt = hcat([1,2,3],[4,5,6])'
tt'

3×2 Matrix{Int64}:
 1  4
 2  5
 3  6

In [36]:
dup_use_cols = Dict{String, Vector}()
dup_use_cols["iy"] = []

Any[]

In [37]:
 # 1329         0 In[39]                    18 long2fix(df_long::DataFrame, tb...
 #  1195         0 In[39]                    21 long2fix(df_long::DataFrame, tb...
 #  1225         0 In[39]                    24 long2fix(df_long::DataFrame, tb...
 #    52         0 In[39]                    28 long2fix(df_long::DataFrame, tb...
 #   130         0 In[39]                    29 long2fix(df_long::DataFrame, tb...
 #   109         0 In[39]                    33 long2fix(df_long::DataFrame, tb...
 #   176         0 In[39]                    37 long2fix(df_long::DataFrame, tb...
 #  1811         0 In[39]                    44 long2fix(df_long::DataFrame, tb...

In [38]:
@everywhere function get_def_cancers(d)
    dis = d["skwd_name_cd"]
    is_def = d["sspct_dss_flg"] .== "0"
    # print(is_def)
    # is_cancer = d["skwd_name_cd"] in Set(["1519006", "1519006"])
    is_cancer = [di in dis_codes for di in dis]
    # print(is_cancer)
    
    def_cancers = dis[is_def .& is_cancer]
    
    return def_cancers
end

In [39]:
@everywhere function get_gen_age(d)
    # from re
    gen = d["sex_div"]
    age = d["age_hier_cd1"]
    return gen, age
end

In [40]:
# tes = Dict{Tuple{String, String}, String}()


In [41]:
const s_dpc_ko_only = Set(["1211","1221","1231","1241"])
const s_med_ko_only = Set(["1211","1212","1221","1222","1231","1232","1241","1242"])
const s_pha_ko_only = Set(["4212","4222","4232","4242"])
const d_ko_only = Dict{String, Set{String}}(
    "dpc"=> s_dpc_ko_only,
    "med"=> s_med_ko_only,
    "pha"=> s_pha_ko_only
)


Dict{String, Set{String}} with 3 entries:
  "dpc" => Set(["1221", "1241", "1231", "1211"])
  "med" => Set(["1232", "1212", "1221", "1241", "1222", "1231", "1211", "1242"])
  "pha" => Set(["4232", "4212", "4242", "4222"])

In [42]:
@everywhere using Mongoc
@everywhere using Dates
@everywhere using DataFrames
@everywhere using DataFramesMeta
@everywhere using JLD2
@everywhere col_days = $col_days
@everywhere need_cols = $need_cols
@everywhere ss_need_cols = $ss_need_cols
@everywhere cols_days_s = $cols_days_s
@everywhere tb_short = $tb_short
@everywhere need_cols =$need_cols
@everywhere tb_dis = $tb_dis
@everywhere tb_re =$tb_re
@everywhere tb3_pha = $tb3_pha
@everywhere tab_codes = $tab_codes
@everywhere dis_codes = $dis_codes 
@everywhere d_ko_only = $d_ko_only

In [43]:
# e
# ex = [i for i in e["dat1"][3]["dat2"] if i["tb"] =="iy"] 

In [44]:
@everywhere function dl2ld(dl::Dict{String, Vector{Union{String, Nothing,Float64}}})
    # Vector{Union{String, Nothing,Float64}}
# function dl2ld(dl::Dict{String, Base.RefValue{Vector{String}}})    
    keys_list = collect(keys(dl))
    #trans for caschening
    values_list = hcat(values(dl)...)
    # values_list = permutedims(hcat(values(dl)...), [2,1])
    n_list, n_keys = size(values_list)
    # n_keys,n_list = size(values_list)
    # return [Dict{String, String}(keys_list[i] => values_list[j,i] for i in 1:n_keys) for j in 1:n_list]
    return [Dict{String, Any}(keys_list[i] => values_list[j,i] for i in 1:n_keys) for j in 1:n_list]
    # return [Dict{String, String}(keys_list[i] => values_list[i,j] for i in 1:n_keys) for j in 1:n_list]
    # return map(Dict, dl)
end


@everywhere function ld2dl(ld::Vector{Dict{String, Any}})
    l_size = length(ld)
    dl = Dict(k => Vector(undef, l_size) for k in keys(ld[1]))
    for (i,d) in enumerate(ld)
        for (k,v) in d
            dl[k][i] =v
        end
    end
    # return d
    return dl
end

In [45]:
l_nan3 = ["nan", NaN, nothing]
const nan3 = Set(l_nan3)

Set{Any} with 3 elements:
  NaN
  nothing
  "nan"

In [46]:
@everywhere nan3 = $nan3

In [47]:
#func create new dict with new key val, "days":kd, "value":kv
@everywhere function add_d(d, kd, kv)
    d_c = copy(d)
    d_c["days"] = kd
    d_c["value"] = kv
    return d_c
end

# function get_long(d, sch, tb)
@everywhere function get_long(dl)
    #input is dict of list
    #make new cols
    # s is cols for days as const vals
    # complement = true
    complement = true
    all_na =true
    
    d_days = Dict(k => dl[k] for k in cols_days_s)
    d_meta = Dict(k => dl[k] for k in setdiff(keys(dl),cols_days_s))

    #to list of dict
    ld_days = dl2ld(d_days)
    ld_meta = dl2ld(d_meta)
    
    
    n_ld_new = 0
    # ld_d = Dict{Int16, Vector}()
    l_ld = []

    #this for loop create long table from  each wide row
    for (i,(di_days, di_meta)) in enumerate(zip(ld_days, ld_meta))
        
        #kd is like "days13_info". kd[5:6] is "13" in this example.
        # ld_din =[add_d(di_meta, kd[5:6], kv) for (kd, kv) in di_days if kv != "nan"]
        # ld_din =[add_d(di_meta, kd[5:6], kv) for (kd, kv) in di_days if kv != nothing && kv != NaN]
        ld_din =[add_d(di_meta, kd[5:6], kv) for (kd, kv) in di_days if kv ∉ nan3]
        
        if length(ld_din) != 0
            push!(l_ld, ld_din)
            
        else
            # ym 2011~2013...? there are records in iy, to, si without dayinfo. they will be days00_info and value as times in long format.
            push!(l_ld, [add_d(di_meta, "00", di_meta["times"])])
        end
        
    end

    res = reduce(vcat, l_ld)
    dl_new = ld2dl(res)
    return dl_new
    # return ld_d
end

In [48]:
# function d2skvvd(d::Dict{String, Vector{String}})
@everywhere function d2skvvd(d::Dict{Any, Any})    
    # # return string key value vector dict
    ## Nan is Float64
    # d2 = Dict{String, Vector{String}}(k => [(if i===NaN || i===nothing "nan" else i end) for i in v] for (k,v) in d)
    d2 = Dict{String, Vector{Union{String, Nothing,Float64}}}(k => v for (k,v) in d if k in ss_need_cols)
    # d = cast(d, Dict{String, Vector{String}})
    # for (k, v) in d
    #     d[k] =[(if i===NaN || i===nothing "nan" else i end) for i in v]
    #     # d2[k] =[(if i===NaN || i===nothing "nan" else i end) for i in v]
        
    #     # if any((x-> x===NaN || x===nothing), v)
            
    #         # d[k]::Vecotr{String} = ["nan" for i in v]
    #     # end
    # end
    return d2
    # return d2
    # return Dict{String, Vector}(d)
end

In [49]:
@everywhere function long2flg(df_long, tb)
    if tb == "iy"
        # println("iy")
        res = @chain df_long  begin
            # @transform(:tag = string.(:days, "_", :medicine_cd, "_", :use_amnt, "_", :value, "_", :aft_splmt_prac_ident))
            @transform(:tag = string.(:days, "_", :medicine_cd, "_", get_str(:use_amnt), "_", get_str(:value), "_", :aft_splmt_prac_ident))
            @transform(:usecode = :medicine_cd)
            @transform(:useqnt_num = get_nums(:use_amnt))
            @transform(:times_num = get_nums(:value))
            @transform(:total_dose = ifelse.([v in tab_codes for v in :medicine_cd],
                    ceil.(:useqnt_num .* :times_num),
                    ceil.(:useqnt_num) .* :times_num))
        end
    elseif tb == "si"
        # println("si")
        res = @chain df_long begin
            @transform(:tag = string.(:days, "_", :prac_act_cd, "_", get_str(:qnt_dat), "_", get_str(:value), "_", :aft_splmt_prac_ident))
            @transform(:usecode = :prac_act_cd)
            @transform(:useqnt_num = get_nums(:qnt_dat))
            @transform(:times_num = get_nums(:value))
            @transform(:total_dose = ceil.(:useqnt_num .* :times_num))
        end
    elseif tb == "to"
        # println("to")
        res = @chain df_long begin
            @transform(:tag = string.(:days, "_", :spcfc_dev_cd, "_", get_str(:use_amnt), "_",get_str(:value), "_", :aft_splmt_prac_ident))
            @transform(:usecode = :spcfc_dev_cd)
            @transform(:useqnt_num = get_nums(:use_amnt))
            @transform(:times_num = get_nums(:value))
            # to is not ceiled
            @transform(:total_dose = :useqnt_num .* :times_num)
        end
    # else
    #     res = df_long
    end
    res = @chain res begin
            @subset(:usecode .!= nothing)
            groupby(:tag)
            @transform(:groupind = string.(eachindex(:tag)))
            @transform(:dup_check = string.(:tag, "_",:groupind))
            @transform(:days_fix = ifelse.(:days .== "00", "15", :days))
    end
    return res
end

In [50]:
@everywhere function long2fix(df_long, tb)
    if tb == "iy"
        res = @chain df_long  begin
            @transform(:usecode = :medicine_cd)
            @transform(:useqnt_num = get_nums(:use_amnt))
            @transform(:times_num = get_nums(:value))
            @transform(:total_dose = ifelse.([v in tab_codes for v in :medicine_cd],
                    ceil.(:useqnt_num .* :times_num),
                    ceil.(:useqnt_num) .* :times_num))
        end
    elseif tb == "si"
        res = @chain df_long begin
            @transform(:usecode = :prac_act_cd)
            @transform(:useqnt_num = get_nums(:qnt_dat))
            @transform(:times_num = get_nums(:value))
            @transform(:total_dose = ceil.(:useqnt_num .* :times_num))
        end
    elseif tb == "to"
        res = @chain df_long begin
            @transform(:usecode = :spcfc_dev_cd)
            @transform(:useqnt_num = get_nums(:use_amnt))
            @transform(:times_num = get_nums(:value))
            # to is not ceiled
            @transform(:total_dose = :useqnt_num .* :times_num)
            
        end
    # else
    #     res = df_long
    end
    res = @chain res begin
            @subset(:usecode .!= nothing)
            @transform(:days_fix = ifelse.(:days .== "00", "15", :days))
    end
    return res
end

In [51]:
# function get_nums(colconv::Vector{Union{String, Nothing,Float64}})
@everywhere function s2n(s)
    # print(string(typeof(s)) * ":")
    # println(s)
    if typeof(s) != Float64
        return parse(Float64, s)
    else
        return s
    end
end

# 
@everywhere function get_nums(colconv::Vector{Union{Missing, String, Nothing,Float64}})
    #println(typeof(colconv[1]))
    # println(length(colconv))
    return [(if i=="nan" || i==nothing || i == NaN 1.0 else s2n(i) end) for i in colconv]
end

@everywhere function get_nums(colconv::Vector{Union{String, Nothing,Float64}})
    #println(typeof(colconv[1]))
    # println(length(colconv))
    return [(if i=="nan" || i==nothing || i == NaN 1.0 else s2n(i) end) for i in colconv]
end


@everywhere function get_nums(colconv::Vector{Any})
    
    return [(if i=="nan" || i==nothing || i == NaN 1.0 else s2n(i) end) for i in colconv]
end

@everywhere function get_str(colconv::Vector{Any})
     return [(if i=="nan" || i==nothing || i == NaN "nan" else i end) for i in colconv]
end

In [52]:
@everywhere function long2use(df_long, sch, tb, d_use)
    
    for row in eachrow(df_long)
        if haskey(d_use[tb], row["usecode"])
            if haskey(d_use[tb][row["usecode"]], row["days_fix"])
                d_use[tb][row["usecode"]][row["days_fix"]] += row["total_dose"]
            else
                d_use[tb][row["usecode"]][row["days_fix"]] = row["total_dose"]
            end
        else
            # if row["days_fix"] == nothing
            #     println(df_long)
            # elseif row["total_dose"] == nothing
            #     println(df_long)
            # elseif row["usecode"] == nothing
            #     println(df_long)
            # else
                d_use[tb][row["usecode"]] = Dict(row["days_fix"] => row["total_dose"])
            # end
        end
    end
end

In [53]:
#name :get_ym_info
@everywhere function get_ym_info(e::Mongoc.BSON)
    #e is dict of finded element.
    
    res = Dict( "id1n" => e["id1n"],
                "ym" => parse(Int32, e["ym"]),
                #"cancers" => Set(),
                "cancers" => Vector{String}(),
                "age" => "",
                "gender" => "",
                "totalpoint" => 0.0
    )
    #d1 as list of schema dict
    d1 = e["dat1"]

    # d_tb =Dict{String, DataFrame}()
    # after fix, to dataframe
    d_rhk = Dict{Tuple{String, String}, Any}()
    d_tb = Dict{Tuple{String, String}, DataFrame}()
    # d_tb =Dict{String, Vector}()
    # d_use = Dict{String, Float64}()
    d_use = Dict{String, Dict{String, Dict{String, Float64}}}()
    d_use["iy"] = Dict{String, Dict{String, Float64}}()
    d_use["to"] = Dict{String, Dict{String, Float64}}()
    d_use["si"] = Dict{String, Dict{String, Float64}}()
    
    for ds in d1
        #each schema table dict
        #get dis from dpc
        if ds["sch"] == "dpc"
            sch = ds["sch"]
            d2 = ds["dat2"]
            dup_flgs = Vector{String}()
            
            for dt in d2
                if dt["tb"] == "cd"
                    # println((ds["sch"], dt["tb"]))
                    #d_tb[(ds["sch"], dt["tb"])] = dt["dat3"] |> (x->d2skvvd(x)) |> (x->DataFrame(x)) 
                    sch_tb =@chain dt["dat3"] |>
                        (x->d2skvvd(x)) |>
                        (x->DataFrame(x)) begin
                        @transform(:days = last.(:oprtn_ymd, 2))
                        @subset(:rcpt_comp_sys_rec .!= nothing)
                        @transform(:code1st = first.(:rcpt_comp_sys_rec))
                        @transform(:code_useqnt = [(if i===nothing || i == NaN "nan" else i end) 
                                for i in ifelse.(:code1st .== '1', :qnt_dat, :use_amnt)])
                        ## here need seq2no
                        @transform(:tag = string.(:days, "_", :rcpt_comp_sys_rec, "_", :code_useqnt, "_", :times, "_", :prac_ident))
                        groupby(:tag)
                        @transform(:groupind = string.(eachindex(:tag)))
                        @transform(:dup_check_code = string.(:tag, "_",:groupind))
                        @transform(:usecode = :rcpt_comp_sys_rec)
                        @transform(:useqnt_num =  [(if i===NaN || i===nothing || i==="nan" 1 else parse(Float64, i) end) for i in :code_useqnt])
                        @transform(:times_num = parse.(Int16, :times))
                        @transform(:total_dose = ceil.(:useqnt_num .* :times_num))
                        @transform(:days_fix = :days)
                    end
                    d_tb[(ds["sch"], dt["tb"])] = sch_tb
                    
                elseif dt["tb"] in tb_short
                    # println((ds["sch"], dt["tb"]))
                    # d_tb[(ds["sch"], dt["tb"])] = DataFrame(dt["dat3"])
                    tbl_pre = 
                    #d_tb[(ds["sch"], dt["tb"])] = tbl_pre |> (x->DataFrame(x))
                    d_tb[(ds["sch"], dt["tb"] * "_long")] = dt["dat3"] |>
                        (x->d2skvvd(x))  |>
                        (x->get_long(x)) |> 
                        (x->DataFrame(x)) |> 
                        (x->long2flg(x, dt["tb"]))
                    dup_flgs = vcat(dup_flgs, d_tb[(ds["sch"], dt["tb"] * "_long")].dup_check)
                    
                    long2use(d_tb[(ds["sch"], dt["tb"] * "_long")], 
                        ds["sch"], dt["tb"], d_use)

                elseif dt["tb"] == "sy"
                    # res["cancers"] = union(res["cancers"],get_def_cancers(dt["dat3"]))
                    # push!(res["cancers"],get_def_cancers(dt["dat3"]))
                    res["cancers"] = vcat(res["cancers"],get_def_cancers(dt["dat3"]))
                elseif dt["tb"] == "sb"
                    # res["cancers"] = union(res["cancers"],get_def_cancers(dt["dat3"]))
                    # push!(res["cancers"],get_def_cancers(dt["dat3"]))
                    res["cancers"] = vcat(res["cancers"],get_def_cancers(dt["dat3"]))
                elseif (dt["tb"] == "re") 
                    # println((ds["sch"], dt["tb"]))
                    d_rhk[(ds["sch"], dt["tb"])] = dt["dat3"]
                    if (res["gender"] == "") & (res["age"] == "")
                        gens, ages = get_gen_age(dt["dat3"])
                        #pick first
                        res["gender"] = gens[1]
                        res["age"] = ages[1]
                    end
                #here hoko
                elseif dt["tb"] == "ho" || dt["tb"] == "ko"
                    # println((ds["sch"], dt["tb"]))
                    d_rhk[(ds["sch"], dt["tb"])] = dt["dat3"]
                else
                    continue
                end
            end
                #d_tb[(ds["sch"], "cd")]
            if haskey(d_tb, ("dpc", "cd"))
                dup_flgs_set = Set(dup_flgs)
                d_tb[("dpc", "cd")] = @transform(d_tb[("dpc", "cd")], :dup_flg2 = [i in dup_flgs_set for i in :dup_check_code])
                d_cd_iy = @subset(d_tb[("dpc", "cd")], :code1st .== '6', :dup_flg2 .== false)
                long2use(d_cd_iy,"dpc", "iy", d_use)
                
                d_cd_si = @subset(d_tb[("dpc", "cd")], :code1st .== '1', :dup_flg2 .== false)
                long2use(d_cd_si,"dpc", "si", d_use)
                
                d_cd_to = @subset(d_tb[("dpc", "cd")], :code1st .== '7', :dup_flg2 .== false)
                long2use(d_cd_to,"dpc", "to", d_use)
            end
            
        elseif ds["sch"] == "med"
            d2 = ds["dat2"]
            for dt in d2
                if dt["tb"] in tb_short
                    # println((ds["sch"], dt["tb"]))
                    tbl_pre = dt["dat3"] |> (x->d2skvvd(x))
                    #d_tb[(ds["sch"], dt["tb"])] = tbl_pre |> (x->DataFrame(x))
                    d_tb[(ds["sch"], dt["tb"] * "_long")] = dt["dat3"] |>
                        (x->d2skvvd(x)) |>
                        (x->get_long(x)) |> 
                        (x->DataFrame(x)) |> 
                        (x->long2fix(x, dt["tb"]))
                    long2use(d_tb[(ds["sch"], dt["tb"] * "_long")], 
                        ds["sch"], dt["tb"], d_use)
                elseif dt["tb"] == "sy"
                    #res["dis_dicts"]["med_sy"] = dt["dat3"]
                    res["cancers"] = vcat(res["cancers"],get_def_cancers(dt["dat3"]))
                    # res["cancers"] = union(res["cancers"],get_def_cancers(dt["dat3"]))
                    # push!(res["cancers"],get_def_cancers(dt["dat3"]))
                    #only sy so break
                elseif (dt["tb"] == "re") 
                    # println((ds["sch"], dt["tb"]))
                    d_rhk[(ds["sch"], dt["tb"])] = dt["dat3"]
                    if (res["gender"] == "") & (res["age"] == "")
                    # if have gen, age, not need this
                    gens, ages = get_gen_age(dt["dat3"])
                    #pick first
                    res["gender"] = gens[1]
                    res["age"] = ages[1]
                    end
                elseif dt["tb"] == "ho" || dt["tb"] == "ko"
                    # println((ds["sch"], dt["tb"]))
                    d_rhk[(ds["sch"], dt["tb"])] = dt["dat3"]
                else
                    continue
                end
            end
        else ## this is not needed because if pha only, no cancer dis name is in.
            # ds["sch"] == "pha" iy dont have day info
            # but need create iy_long as usage info
            d2 = ds["dat2"]
        
            # here we need combine pha iy, sh, cz
            # I call it iy_long
            # not yet implmented
            # d_tb[(ds["sch"], "iy_long")] = get_pha_iy_long(d2)
            for dt in d2
                if dt["tb"] in tb3_pha
                    # println((ds["sch"], dt["tb"]))
                    d_tb[(ds["sch"], dt["tb"])] = dt["dat3"] |> (x->d2skvvd(x))|> (x->DataFrame(x))
                    # d_tb[(ds["sch"], dt["tb"] * "_long")] = d_tb[(ds["sch"], dt["tb"])] |> (x->get_long(x)) 
                    # print(dt["dat3"])
                elseif (dt["tb"] == "re") 
                    # println((ds["sch"], dt["tb"]))
                    d_rhk[(ds["sch"], dt["tb"])] = dt["dat3"]
                    if (res["gender"] == "") & (res["age"] == "")
                        # if have gen, age, not need this
                        gens, ages = get_gen_age(dt["dat3"])
                        #pick first
                        res["gender"] = gens[1]
                        res["age"] = ages[1]
                    end
                elseif dt["tb"] == "ho" || dt["tb"] == "ko"
                    # println((ds["sch"], dt["tb"]))
                    d_rhk[(ds["sch"], dt["tb"])] = dt["dat3"]
                else
                    continue
                end
            end

            # here prspt_ymd -> :dspng_ymd
            if haskey(d_tb, ("pha", "iy")) && haskey(d_tb, ("pha", "cz"))
                df_pha_iy = @select(d_tb[("pha", "iy")], :medicine_cd, :prspt_no, :use_amnt, :dose)
                df_pha_cz = @select(d_tb[("pha", "cz")], :dspng_ymd, :dspng_amnt, :prspt_no)
                d_tb[(ds["sch"], "iy_long")] = @chain leftjoin(df_pha_iy, df_pha_cz, on=:prspt_no) begin
                    @transform(:days_fix = last.(:dspng_ymd, 2))
                    @transform(:usecode = :medicine_cd)
                    @subset(:usecode .!= nothing)
                    # use_amnt from iy, dose from iy, DSPNG_AMNT from cz
                    @transform(:useqnt_num = get_nums(:use_amnt))
                    @transform(:dose_num = get_nums(:dose))
                    @transform(:dspng_amnt = get_nums(:dspng_amnt))
                    @transform(:total_dose = ifelse.([v in tab_codes for v in :medicine_cd],
                            ceil.(:useqnt_num .* :dose_num .* :dspng_amnt),
                            ceil.(:useqnt_num .* :dose_num) .* :dspng_amnt))
                end

                long2use(d_tb[(ds["sch"], "iy_long")], 
                        ds["sch"], "iy", d_use)
            end
        end
    end
    res["usage"] = d_use
    res["totalpoint"] = get_point(d_rhk)
    unique!(res["cancers"])
    # return res, d_tb, d_use
    return res
end

In [54]:
# Profile.clear()
# # @profile xtest = db2hdf("d_001",10000)
# @profile xtest = db2hdf("d_001",1000)
# print(length(xtest))

In [55]:
# Profile.print(mincount = 100)

In [56]:
# Profile.print(format=:flat, mincount = 1000, )

In [57]:
# # es[1]
# _, d_tb, _ = get_ym_info(es[7])
# length(d_tb)

In [58]:
@everywhere function get_ko_seq2no(d, sch)
    d_re = d[sch, "re"]
    # println(d_re)
    # return [d["seq2_no"] for d in d_re if d["rcp_cls"] in d_ko_only[sch]]
    return [seq2_no for (seq2_no, rcp_cls)  in zip(d_re["seq2_no"], d_re["rcp_cls"]) if rcp_cls in d_ko_only[sch]]
end

In [59]:
@everywhere function parse_f(score)
    if score in nan3
        return 0.0
    else
        return parse(Float64, score)
    end
end

In [60]:
@everywhere function get_yen_ho(d, sch, ko_only_seq2)
    d_ho = d[sch, "ho"]
    # some time total score will be nothing even if seq2_no not in ko_only
    return [parse_f(total_score) for (seq2_no, total_score) in zip(d_ho["seq2_no"],d_ho["total_score"]) if !(seq2_no in ko_only_seq2)]
    # return [parse(Float64, total_score) for (seq2_no, total_score) in zip(d_ho["seq2_no"],d_ho["total_score"]) if !(seq2_no in ko_only_seq2)]
end

@everywhere function get_yen_ko(d, sch, ko_only_seq2)
    d_ko = d[sch, "ko"]
    return [parse_f(total_score) for (seq2_no, total_score) in zip(d_ko["seq2_no"],d_ko["total_score"]) if seq2_no in ko_only_seq2]
    # return [parse(Float64, total_score) for (seq2_no, total_score) in zip(d_ko["seq2_no"],d_ko["total_score"]) if seq2_no in ko_only_seq2]
end

In [61]:
@everywhere function get_point(d_hrk)
    total = 0

    for sch in ["med", "dpc", "pha"]
        if haskey(d_hrk, (sch, "re"))
            # print("med")
            seq2 = get_ko_seq2no(d_hrk, sch)
            if length(seq2) !=0
                l_yen_ko = get_yen_ko(d_hrk, sch, seq2)
                total += sum(l_yen_ko)
                # println(l_yen_ko)
            end
            if haskey(d_hrk, (sch, "ho"))
                l_yen_ho = get_yen_ho(d_hrk, sch, seq2)
                total += sum(l_yen_ho)
            end
            # println(l_yen_ho)
        end
    end
    return total
end

In [62]:
dbs[1:3]

3-element Vector{String}:
 "d_000"
 "d_001"
 "d_002"

In [63]:
@everywhere a = now()

@everywhere function db2hdf(dbi, limit_num)
    client = Mongoc.Client("192.168.0.25", 27017)
    db = client["ndb"]
    collection = db[dbi]
    
    sd = client["smr"]
    sdi = replace(dbi, "d_" =>"s_")
    smr_col = sd[sdi]
    
    a2 = now()
    
    l = Vector{Mongoc.BSON}()
    # i = 0
    # for (i,ei) in enumerate(collection)
    for (i, ei) in enumerate(Mongoc.find(collection))
        # i += 1
        # println(i)
        # println(e)

        # res = get_ym_info(ei)
        push!(l,Mongoc.BSON(get_ym_info(ei)))
    
        # limitation for row here
        # if i > limit_num
        #     break
        # end 
        
        if i % 1e+4 == 0
            append!(smr_col, l)
            l = Vector{Mongoc.BSON}()

            if i % 1e+6 == 0
                print("db:")
                print(dbi)
                print(", checkpoint:")
                print(i)
                b = now()
        
                print(" time:")
                print(canonicalize(b -a))
                print(" chunktime:")
                print(canonicalize(b -a2))
                # print(" dictsize:")
                # print(length(d_id1n))
                print("\n")
                a2 = now()
            end

        end

        # if i == 1e+4
        #     break
        # end
    end

    
    # ts = replace(string(a),"-"=>"", "T"=>"_", ":"=> "", "."=> "")
    # # res_pf = "./res/res_$(dbi)_$(ts).jld2"
    # res_pf = "../../../storage/res_temp/res_$(dbi)_$(ts).jld2"
    # # res_pf
    # println(res_pf)
    # # jldsave(res_pf; l= l)
    # # save(res_pf, compress = true; l= l)
    # # save(res_pf,l= l, compression="gzip")
    # # return l
    return []
    
end

In [64]:
@everywhere function tests(dbi)
    db2hdf(dbi, 100000000)
    # db2hdf(dbi, 10000)
    return []
end


In [65]:
dbs[1:128]

128-element Vector{String}:
 "d_000"
 "d_001"
 "d_002"
 "d_003"
 "d_004"
 "d_005"
 "d_006"
 "d_007"
 "d_008"
 "d_009"
 "d_010"
 "d_011"
 "d_012"
 ⋮
 "d_116"
 "d_117"
 "d_118"
 "d_119"
 "d_120"
 "d_121"
 "d_122"
 "d_123"
 "d_124"
 "d_125"
 "d_126"
 "d_127"

In [66]:
dbs[129:256]

128-element Vector{String}:
 "d_128"
 "d_129"
 "d_130"
 "d_131"
 "d_132"
 "d_133"
 "d_134"
 "d_135"
 "d_136"
 "d_137"
 "d_138"
 "d_139"
 "d_140"
 ⋮
 "d_244"
 "d_245"
 "d_246"
 "d_247"
 "d_248"
 "d_249"
 "d_250"
 "d_251"
 "d_252"
 "d_253"
 "d_254"
 "d_255"

In [67]:
# tests("d_022")

In [68]:


# a = now()

# results = @distributed (+) for e in [x for x in collection]
# results = @distributed (+) for e in collect(cursor)
#     get_id1ym_dis(e)
# end


# results = @distributed (+) for dbxs in dbs[1:2]
results = @distributed (+) for dbxs in dbs[1:128]
    # get_d(dbxs)
    tests(dbxs)
    
    # println(dbxs)
end

# b = now()
# print("\n")
# print(canonicalize(b -a))

      From worker 16:	db:d_056, checkpoint:1000000 time:38 minutes, 45 seconds, 850 milliseconds chunktime:38 minutes, 32 seconds, 488 milliseconds
      From worker 5:	db:d_012, checkpoint:1000000 time:39 minutes, 14 seconds, 881 milliseconds chunktime:39 minutes, 1 second, 129 milliseconds
      From worker 9:	db:d_028, checkpoint:1000000 time:39 minutes, 19 seconds, 998 milliseconds chunktime:39 minutes, 6 seconds, 57 milliseconds
      From worker 21:	db:d_076, checkpoint:1000000 time:39 minutes, 24 seconds, 188 milliseconds chunktime:39 minutes, 10 seconds, 347 milliseconds
      From worker 30:	db:d_112, checkpoint:1000000 time:39 minutes, 29 seconds, 57 milliseconds chunktime:39 minutes, 15 seconds, 639 milliseconds
      From worker 19:	db:d_068, checkpoint:1000000 time:39 minutes, 34 seconds, 144 milliseconds chunktime:39 minutes, 20 seconds, 564 milliseconds
      From worker 12:	db:d_040, checkpoint:1000000 time:39 minutes, 40 seconds, 302 milliseconds chunktime:39 minutes, 

Any[]

need restart mongo

In [69]:
results = @distributed (+) for dbxs in dbs[129:256]
    # get_d(dbxs)
    tests(dbxs)
    
    # println(dbxs)
end

      From worker 5:	db:d_140, checkpoint:1000000 time:2 days, 3 hours, 56 minutes, 40 seconds, 925 milliseconds chunktime:37 minutes, 12 seconds, 433 milliseconds
      From worker 18:	db:d_192, checkpoint:1000000 time:2 days, 3 hours, 56 minutes, 42 seconds, 284 milliseconds chunktime:37 minutes, 13 seconds, 793 milliseconds
      From worker 7:	db:d_148, checkpoint:1000000 time:2 days, 3 hours, 56 minutes, 53 seconds, 842 milliseconds chunktime:37 minutes, 25 seconds, 352 milliseconds
      From worker 16:	db:d_184, checkpoint:1000000 time:2 days, 3 hours, 56 minutes, 59 seconds, 244 milliseconds chunktime:37 minutes, 30 seconds, 759 milliseconds
      From worker 3:	db:d_132, checkpoint:1000000 time:2 days, 3 hours, 57 minutes, 2 seconds, 86 milliseconds chunktime:37 minutes, 33 seconds, 597 milliseconds
      From worker 13:	db:d_172, checkpoint:1000000 time:2 days, 3 hours, 57 minutes, 5 seconds, 472 milliseconds chunktime:37 minutes, 36 seconds, 979 milliseconds
      From worke

Any[]

In [1]:
# kokomade